# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [11]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,74.82,73,13,14.90,few clouds
1,1,Eravur,LK,7.7733,81.6095,76.50,85,100,11.39,overcast clouds
2,2,Belaya Gora,RU,68.5333,146.4167,-46.77,92,36,3.02,scattered clouds
3,3,Cuamba,MZ,-14.8031,36.5372,71.76,85,92,2.42,light rain
4,4,Thompson,CA,55.7435,-97.8558,-27.24,75,20,5.75,few clouds
5,5,Araouane,ML,18.9048,-3.5265,83.84,13,81,5.93,broken clouds
6,6,Tiksi,RU,71.6872,128.8694,-23.80,100,82,4.27,broken clouds
7,7,Castro,BR,-24.7911,-50.0119,75.43,77,47,8.93,scattered clouds
8,8,Kashi,CN,39.4547,75.9797,29.41,34,63,2.24,broken clouds
9,9,Atuona,PF,-9.8000,-139.0333,77.05,77,46,14.27,light rain


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [12]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [13]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,74.82,73,13,14.90,few clouds
1,1,Eravur,LK,7.7733,81.6095,76.50,85,100,11.39,overcast clouds
3,3,Cuamba,MZ,-14.8031,36.5372,71.76,85,92,2.42,light rain
5,5,Araouane,ML,18.9048,-3.5265,83.84,13,81,5.93,broken clouds
7,7,Castro,BR,-24.7911,-50.0119,75.43,77,47,8.93,scattered clouds
9,9,Atuona,PF,-9.8000,-139.0333,77.05,77,46,14.27,light rain
13,13,Comodoro Rivadavia,AR,-45.8667,-67.5000,73.26,21,40,21.85,scattered clouds
17,17,Hilo,US,19.7297,-155.0900,75.27,96,100,4.00,moderate rain
18,18,Faanui,PF,-16.4833,-151.7500,79.65,75,19,17.40,few clouds
20,20,Meulaboh,ID,4.1363,96.1285,73.76,95,100,3.56,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [14]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,74.82,73,13,14.90,few clouds
1,1,Eravur,LK,7.7733,81.6095,76.50,85,100,11.39,overcast clouds
3,3,Cuamba,MZ,-14.8031,36.5372,71.76,85,92,2.42,light rain
5,5,Araouane,ML,18.9048,-3.5265,83.84,13,81,5.93,broken clouds
7,7,Castro,BR,-24.7911,-50.0119,75.43,77,47,8.93,scattered clouds
9,9,Atuona,PF,-9.8000,-139.0333,77.05,77,46,14.27,light rain
13,13,Comodoro Rivadavia,AR,-45.8667,-67.5000,73.26,21,40,21.85,scattered clouds
17,17,Hilo,US,19.7297,-155.0900,75.27,96,100,4.00,moderate rain
18,18,Faanui,PF,-16.4833,-151.7500,79.65,75,19,17.40,few clouds
20,20,Meulaboh,ID,4.1363,96.1285,73.76,95,100,3.56,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
0,Rikitea,PF,74.82,few clouds,-23.1203,-134.9692
1,Eravur,LK,76.50,overcast clouds,7.7733,81.6095
3,Cuamba,MZ,71.76,light rain,-14.8031,36.5372
5,Araouane,ML,83.84,broken clouds,18.9048,-3.5265
7,Castro,BR,75.43,scattered clouds,-24.7911,-50.0119
9,Atuona,PF,77.05,light rain,-9.8000,-139.0333
13,Comodoro Rivadavia,AR,73.26,scattered clouds,-45.8667,-67.5000
17,Hilo,US,75.27,moderate rain,19.7297,-155.0900
18,Faanui,PF,79.65,few clouds,-16.4833,-151.7500
20,Meulaboh,ID,73.76,overcast clouds,4.1363,96.1285


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [16]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,74.82,few clouds,-23.1203,-134.9692,
1,Eravur,LK,76.50,overcast clouds,7.7733,81.6095,
3,Cuamba,MZ,71.76,light rain,-14.8031,36.5372,
5,Araouane,ML,83.84,broken clouds,18.9048,-3.5265,
7,Castro,BR,75.43,scattered clouds,-24.7911,-50.0119,
9,Atuona,PF,77.05,light rain,-9.8000,-139.0333,
13,Comodoro Rivadavia,AR,73.26,scattered clouds,-45.8667,-67.5000,
17,Hilo,US,75.27,moderate rain,19.7297,-155.0900,
18,Faanui,PF,79.65,few clouds,-16.4833,-151.7500,
20,Meulaboh,ID,73.76,overcast clouds,4.1363,96.1285,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [17]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [18]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df(10)

Starting hotel search
Rikitea - nearest hotel: Chez Bianca & Benoit
Eravur - nearest hotel: Hotel
Cuamba - nearest hotel: Pensao Cariaco
Araouane - nearest hotel: No hotel found
Castro - nearest hotel: Hotel Central Palace
Atuona - nearest hotel: Pearl Resort
Comodoro Rivadavia - nearest hotel: No hotel found
Hilo - nearest hotel: Dolphin Bay Hotel
Faanui - nearest hotel: Oa Oa Lodge
Meulaboh - nearest hotel: Hotel Meuligou
Kununurra - nearest hotel: Kununurra Country Club Resort
Florianopolis - nearest hotel: Pousada Caminho da Lagoa
Butaritari - nearest hotel: No hotel found
Bonthe - nearest hotel: No hotel found
Parrita - nearest hotel: Hotel Paraíso Carlisa
Cidreira - nearest hotel: Hotel Castelo
Vaini - nearest hotel: Keleti International Resort
Toamasina - nearest hotel: Androranga
Coahuayana - nearest hotel: No hotel found
Hermosillo - nearest hotel: Lucerna
Arraial Do Cabo - nearest hotel: No hotel found
Salalah - nearest hotel: Muscat International Hotel
Toa Alta - nearest hot

Nauta - nearest hotel: Hostal Henry
Tlacolula - nearest hotel: No hotel found
Los Banos - nearest hotel: Palmridge Private Resort
Tshikapa - nearest hotel: Hôtel TSHAMUNTU
Samarai - nearest hotel: No hotel found
Maputo - nearest hotel: No hotel found
Pires Do Rio - nearest hotel: No hotel found
Souillac - nearest hotel: Hotel SkylineInn
Ixtapa - nearest hotel: No hotel found
Moa - nearest hotel: Hotel Miraflores
Mopipi - nearest hotel: No hotel found
Salima - nearest hotel: Palms lodge
Benguela - nearest hotel: Hotel Moibela
Touros - nearest hotel: Pousada Atlântico
Berbera - nearest hotel: Al Madina Hotel
Betioky - nearest hotel: Hotel Salema
Mayumba - nearest hotel: Résidence Hôtelière Fabso
Gewane - nearest hotel: No hotel found
Gouyave - nearest hotel: No hotel found
Rio Verde De Mato Grosso - nearest hotel: No hotel found
Manta - nearest hotel: La Gaviota
Mount Isa - nearest hotel: Ibis Styles
Papar - nearest hotel: No hotel found
Porto Velho - nearest hotel: Hotel Slaviero Essent

TypeError: 'DataFrame' object is not callable

## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [19]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= "No hotel found"]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,74.82,few clouds,-23.1203,-134.9692,Chez Bianca & Benoit
1,Eravur,LK,76.50,overcast clouds,7.7733,81.6095,Hotel
3,Cuamba,MZ,71.76,light rain,-14.8031,36.5372,Pensao Cariaco
7,Castro,BR,75.43,scattered clouds,-24.7911,-50.0119,Hotel Central Palace
9,Atuona,PF,77.05,light rain,-9.8000,-139.0333,Pearl Resort
17,Hilo,US,75.27,moderate rain,19.7297,-155.0900,Dolphin Bay Hotel
18,Faanui,PF,79.65,few clouds,-16.4833,-151.7500,Oa Oa Lodge
20,Meulaboh,ID,73.76,overcast clouds,4.1363,96.1285,Hotel Meuligou
30,Kununurra,AU,80.58,broken clouds,-15.7667,128.7333,Kununurra Country Club Resort
32,Florianopolis,BR,80.55,broken clouds,-27.6146,-48.5012,Pousada Caminho da Lagoa


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [20]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [22]:
# Configure the map plot
map_plot = clean_hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Max Temp",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)